In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0
from kaggle_datasets import KaggleDatasets

# Step 1: Initialize TPU
# try:
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # Detect TPU
#     tf.config.experimental_connect_to_cluster(tpu)
#     tf.tpu.experimental.initialize_tpu_system(tpu)
#     strategy = tf.distribute.TPUStrategy(tpu)
#     print("All TPU devices: ", tf.config.list_logical_devices('TPU'))
# except ValueError:
#     print("TPU not founddd.")

# strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

# Use GPU

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  2


# Dataset 

In [3]:
import keras

In [4]:
image_size = (224, 224)
batch_size = 128

train_ds, val_ds = keras.utils.image_dataset_from_directory(
    "/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train",
    validation_split=0.2,
    subset="both",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

Found 100000 files belonging to 2 classes.
Using 80000 files for training.
Using 20000 files for validation.


In [5]:
with strategy.scope():
    model = tf.keras.Sequential([
        EfficientNetB0(include_top=False, input_shape=(224, 224, 3), pooling='avg'),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

# Step 4: Train Model
epochs = 100
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    steps_per_epoch=len(train_ds) // batch_size,
    validation_steps=len(val_ds) // batch_size
)

# Step 5: Save Model
model.save("efficientnet_cifake.h5")

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/100


2024-12-08 16:31:29.888267: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/cond/else/_1550/cond/StatefulPartitionedCall/replica_1/sequential_1/efficientnetb0_1/block2b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


4/4 ━━━━━━━━━━━━━━━━━━━━ 127s 3s/step - accuracy: 0.6509 - loss: 0.6245 - val_accuracy: 0.6250 - val_loss: 0.6178
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 730ms/step - accuracy: 0.8784 - loss: 0.3208 - val_accuracy: 0.5781 - val_loss: 0.6882
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 717ms/step - accuracy: 0.8707 - loss: 0.2961 - val_accuracy: 0.6562 - val_loss: 0.6201
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 727ms/step - accuracy: 0.9060 - loss: 0.2896 - val_accuracy: 0.7344 - val_loss: 0.8132
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 742ms/step - accuracy: 0.9322 - loss: 0.1850 - val_accuracy: 0.6406 - val_loss: 1.3505
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 731ms/step - accuracy: 0.8952 - loss: 0.2262 - val_accuracy: 0.5312 - val_loss: 1.5332
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 721ms/step - accuracy: 0.9039 - loss: 0.2734 - val_accuracy: 0.6094 - val_loss: 1.4707
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 734ms/step - accuracy: 0.9198 - loss: 0.1792 - val_accuracy: 0.6250 - val_loss: 1.27

# On Test Dataset

In [6]:
test_ds = keras.utils.image_dataset_from_directory(
    "/kaggle/input/cifake-real-and-ai-generated-synthetic-images/test",
    image_size=image_size,
    batch_size=batch_size,
)
result = model.evaluate(test_ds)
# Print the metrics and their values
for metric_name, metric_value in zip(model.metrics_names, result):
    print(f"{metric_name}: {metric_value}")

Found 20000 files belonging to 2 classes.
157/157 ━━━━━━━━━━━━━━━━━━━━ 32s 202ms/step - accuracy: 0.8345 - loss: 0.4824
loss: 0.31516221165657043
compile_metrics: 0.8787999749183655
